# Harjoitustyö 2: Datan keräys
### *Ilpo Viertola, Johdanto datatieteeseen 2021*

Tässä harjoitustyön raportissa käyn lävitse, kuinka keräsin työssä käytetyn datasetin. Datasetit hain kahdesta eri lähteestä:  
1. Kagglen tarjoama valmis datasetti  
2. Twitter API:n ja Tweepyn avulla itse kerätty datasetti  

## Kagglen tarjoama datasetti
  
Käytetyn datasetin löytää [täältä](https://www.kaggle.com/elvinagammed/covid19-fake-news-dataset-nlp/code). Tämä datasetti koostuu eri sosiaalisen median kanavista kerätyistä englannin kielisistä julkaisuista, jotka sisältävät Koronavirukseen liittyvää informaatiota. Datasetti myös sisältää tiedon siitä, onko julkaisun informaatio paikkaansa pitävää. Tämän datasetin avulla pyrin harjoitustyön seuraavissa vaiheissa opettamaan luokittelijan, joka osaa tunnistaa väärää Koronavirus informaatiota sisältävät julkaisut. Käytän tästä datasetistä pääsääntöisesti kahta eri tiedostoa harjoitustyön seuraavissa vaiheissa:  
  
1. Constraint_Train.csv: Sisältää opetusdatan.  
2. Constraint_Test.csv: Sisältää testidatan.  
  
## Tweepyn avulla kerätty datasetti
  
Käytin Pythonin Tweepy kirjastoa, joka suoraviivaistaa Twitter API:n käyttöä. Twitter API mahdollistaa esimerkiksi twiittien reaaliaikaisen hakemisen Twitteristä ohjelmakoodissa. Tämän kerätyn datasetin avulla pyrin koeponnistamaan tekemääni mallia reaalimaailmassa, eli luokittelemaan satunnaisia twiittejä. Twiitit, joita työssä käytän ovat englannin kielisiä ja sisältävät #covid19-tagin. Näin voidaan olettaa, että twiitin sisältö liittyy edes jotenkin koronaviirukseen. On mielenkiintoista nähdä, miten luokittelija käyttäytyy heikommin kuratoiduille syötteille. Alla on esitettynä Python-koodi, jota twiittien hakemiseen käytin.   

In [13]:
import tweepy
import pandas as pd

# Autentikointi Twitteriin
consumer_key = ''
consumer_secret = ''
access_key = ''
access_key_secret = ''

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_key, access_key_secret)
api = tweepy.API(auth)  # Rajapinta

# Luodaan DataFrame twiiteille
tweet_df = pd.DataFrame(columns=['username', 'description', 'location', 'following', 'followers', 'totaltweets', 'retweetcount', 'text', 'hashtags'])

# Tweepyn Cursor-objektille annetaan parametrina api.search-metodi ja haun parametrit. 
# Rajoitetaan haettavien twiittien määrä esim. 50:neen.
hashtag = '#covid19'
d_since = '2021-04-07'
limit = 50
tweets = tweepy.Cursor(api.search, q=hashtag, lang='en', since=d_since, tweet_mode='extended').items(limit)

# Cursor palauttaa iteroitavat objektin, joten iteroidaan sen sisältö listaan.
tweets_list = [tweet for tweet in tweets]

# Käydään läpi twiitit listasta.
for tweet in tweets_list:
    # Tietoja twiitistä ja twiittaajasta myöhempään tarkasteluun
    username = tweet.user.screen_name
    description = tweet.user.description
    location = tweet.user.location
    following = tweet.user.friends_count
    followers = tweet.user.followers_count
    totaltweets = tweet.user.statuses_count
    retweetcount = tweet.retweet_count
    hashtags = tweet.entities['hashtags']
        
    # Uudelleentwiittaukset voidaan erottaa retweeted_status-määritteellä.
    # Jos twiitti ei ole retwiitti, except-lohko suoritetaan.
    try:
        text = tweet.retweeted_status.full_text
    except AttributeError:
        text = tweet.full_text
    hashtext = list()
    for j in range(0, len(hashtags)):
        hashtext.append(hashtags[j]['text'])
        
    # Lisätään data DataFrameen.
    ith_tweet = [username, description, location, following, followers, totaltweets, retweetcount, text, hashtext]
    tweet_df.loc[len(tweet_df)] = ith_tweet

# Rivien määrä sama kuin asetettu raja.
print('Amount of tweets: ' + str(len(tweet_df)))
# Katsaus muutamaan ensimmäiseen riviin.
tweet_df.head()

Amount of tweets: 50


,username,description,location,following,followers,totaltweets,retweetcount,text,hashtags
0,Sampath1986,"Filmmaker, Indian cinema","Chennai, India",919,161,52645,4,#Lucknow\nNeed #Blood Type : O-negative\nAt :...,"[Lucknow, Blood, COVID19]"
1,DbAshby,Brexit is a disaster for the UK economy #Brexi...,Cheshire,6259,5986,303793,243,"""Vaccines alone will not be able to protect us...",[COVID19]
2,SEEP_Fiji,People. Centred. Development. \nReal DEMOCRACY,"Suva, Fiji",197,241,1273,0,Demystify all the fear mongering around the #C...,"[COVID19, vaccine]"
3,yodisculpe,27 🇲🇽 \nAlways hungry/always tired. Wear a mas...,L.A.,218,14,5964,109,L.A. PEOPLE: Just checked and there are some #...,[COVID19]
4,SOUMYA_PAL24,,,0,0,52,52,Doing online meeting to secure themselves and ...,[Examwarrior]


## Hyödyllisiä linkkejä
1. [Kagglen datasetti](https://www.kaggle.com/elvinagammed/covid19-fake-news-dataset-nlp?select=Constraint_Test.csv)
2. [Tweepyn dokumentaatio](https://docs.tweepy.org/en/latest/index.html)
3. [Geeks4Geeks esimerkki](https://www.geeksforgeeks.org/extracting-tweets-containing-a-particular-hashtag-using-python/)

## Helppoa ja haastavaa
1. Kagglesta hyvän ja ajankohtaisen datasetin etsiminen saattaa vaati hieman aikaa.  
2. Tweepy oli täysin uusi tuttavuus, joten sen omaksuminen otti aikansa.  
3. Twitter API:n datankeräys katot aiheuttivat välillä päänvaivaa. 